# Evaluate the heuristic shot detection algorithm using the dev/test split from the sequential weak supervision NeurIPS 2019 paper

In [1]:
import random
from esper.prelude import *
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.temporal_predicates import *
from esper.rekall import *
import matplotlib.pyplot as plt
import cv2
import pickle

In [18]:
# Load up all shots from the algorithm
shots_qs = Shot.objects.filter(cinematic=True)

In [43]:
shots = VideoIntervalCollection.from_django_qs(shots_qs)

In [33]:
shots_manual_qs = Shot.objects.filter(labeler__name__contains='manual')

In [34]:
shots_manual = VideoIntervalCollection.from_django_qs(shots_manual_qs)

In [7]:
VAL_WINDOWS = '/app/data/shot_detection_weak_labels/validation_windows_same_val_test.pkl'
TEST_WINDOWS = '/app/data/shot_detection_weak_labels/test_windows_same_val_test.pkl'
Y_VAL = '/app/data/shot_detection_weak_labels/Y_val_windows_downsampled_same_val_test.npy'
Y_TEST = '/app/data/shot_detection_weak_labels/Y_test_windows_downsampled_same_val_test.npy'

In [8]:
with open(VAL_WINDOWS, 'rb') as f:
    val_windows_by_video_id = pickle.load(f)
with open(TEST_WINDOWS, 'rb') as f:
    test_windows_by_video_id = pickle.load(f)
with open(Y_VAL, 'rb') as f:
    Y_val = np.load(f)
with open(Y_TEST, 'rb') as f:
    Y_test = np.load(f)

In [26]:
val_windows_collected = collect(val_windows_by_video_id, lambda row: row[0])
val_collection = VideoIntervalCollection({
    video_id: [
        (tup[1], tup[2], 2)
        for tup in val_windows_collected[video_id]
    ]
    for video_id in val_windows_collected
})

In [29]:
test_windows_collected = collect(test_windows_by_video_id, lambda row: row[0])
test_collection = VideoIntervalCollection({
    video_id: [
        (tup[1], tup[2], 2)
        for tup in test_windows_collected[video_id]
    ]
    for video_id in test_windows_collected
})

In [50]:
shot_boundaries = shots.map(
    lambda intrvl: (intrvl.start - 1, intrvl.start, 1)
)
shot_boundaries_gt = shots_manual.map(
    lambda intrvl: (intrvl.start - 1, intrvl.start, 1)
)

In [47]:
predicted_val = val_collection.filter_against(
    shot_boundaries,
    predicate = during_inv()
).map(
    lambda intrvl: (intrvl.start, intrvl.end, 1)
)

In [51]:
gt_val = val_collection.filter_against(
    shot_boundaries_gt,
    predicate = during_inv()
).map(
    lambda intrvl: (intrvl.start, intrvl.end, 1)
)

In [58]:
def size(intrvlcol):
    return sum(
        len(intrvlcol.get_intervallist(video_id).get_intervals())
        for video_id in intrvlcol.get_allintervals()
    )

In [64]:
def compute_prf1(predicted, gt):
    tp = predicted.filter_against(
        gt,
        predicate = lambda i1, i2: i1.start == i2.start and i1.end == i2.end
    )
    fp = predicted.minus(gt)
    fn = gt.minus(predicted)
    
    precision = size(tp) / (size(tp) + size(fp))
    recall = size(tp) / (size(tp) + size(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1, size(tp), size(fp), size(fn)

In [65]:
compute_prf1(predicted_val, gt_val)

(0.9247311827956989, 0.8571428571428571, 0.889655172413793, 258, 21, 43)

In [66]:
predicted_test = test_collection.filter_against(
    shot_boundaries,
    predicate = during_inv()
).map(
    lambda intrvl: (intrvl.start, intrvl.end, 1)
)

In [67]:
gt_test = test_collection.filter_against(
    shot_boundaries_gt,
    predicate = during_inv()
).map(
    lambda intrvl: (intrvl.start, intrvl.end, 1)
)

In [68]:
compute_prf1(predicted_test, gt_test)

(0.8084291187739464, 0.8755186721991701, 0.8406374501992032, 211, 50, 30)